### collaborative filtering:
ALS method

Explicit Ratings and implicit Ratings:



Information of Product and user

Content based: Estimate using some user and some product

Collaborative: Use information about other users and products too( users who buy same product and hence are of similar nature and hence can buy similar other product)...Netflix Movies recommendation

Hybrid method: Mixture of above technique



latent factor analysis

Alternative Least Square method-  weighted regularization

Real World Problem:

Sparsity: Cold Start(New User), Computational Intensity

implicit rating:Browsing history, mouse movement, time spent listening viewing /Hard to use...

"collaborative filtering for implicit Datasets"- Yifan Hu, Yehuda Koren, Chris Volinsky


root mean sq not used often but RANK MEASURE is used

Cold Start: Spark assigns NaN and drops in Cross validation

In [109]:
from pyspark.sql import SparkSession

spark=SparkSession\
.builder\
.appName('Use of collaborative filtering for movie Recommendation System')\
.getOrCreate()

rawData=spark.read\
.format('csv')\
.option('header','true')\
.load("/home/titan/dataset/ML_JRVI/movielens/ratings.csv")

In [110]:
rawData.toPandas().head(3)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182


In [111]:
from pyspark.sql.functions import col
dataset=rawData.select(col('userID').cast('float'),
                      col('movieID').cast('float'),
                      col('rating').cast('float'),
                      )

dataset.toPandas().head()

,userID,movieID,rating
0,1.0,31.0,2.5
1,1.0,1029.0,3.0
2,1.0,1061.0,3.0
3,1.0,1129.0,2.0
4,1.0,1172.0,4.0


In [112]:
dataset.count()

100004

In [113]:
dataset=dataset.replace('?',None).dropna(how='any')
dataset.count()

100004

In [114]:
dataset.select('rating').toPandas().describe()

,rating
count,100004.000000
mean,3.543608
std,1.058048
min,0.500000
25%,3.000000
50%,4.000000
75%,4.000000
max,5.000000


In [115]:
(trainingData,testData)=dataset.randomSplit([0.8,0.2])

In [116]:
from pyspark.ml.recommendation import ALS

als=ALS(maxIter=5,
       regParam=0.1,
       userCol='userID',
       itemCol='movieID',
       ratingCol='rating',
       coldStartStrategy='drop')

In [117]:
model=als.fit(trainingData)

In [118]:
prediction=model.transform(testData)
prediction.toPandas().head(3)

,userID,movieID,rating,prediction
0,232.0,463.0,4.0,4.022904
1,452.0,463.0,2.0,2.792961
2,126.0,471.0,5.0,3.977929


In [119]:
prediction.select('rating','prediction').toPandas().describe()

,rating,prediction
count,19261.000000,19261.000000
mean,3.562224,3.383614
std,1.051597,0.752642
min,0.500000,-0.020539
25%,3.000000,2.946074
50%,4.000000,3.464280
75%,4.000000,3.914453
max,5.000000,5.599305


In [120]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator=RegressionEvaluator(
labelCol='rating',
predictionCol='prediction',
metricName='r2')

rsquare=evaluator.evaluate(prediction)
print("Test r2 score = %g"% rsquare)

Test r2 score = 0.229353


In [121]:
evaluator=RegressionEvaluator(
labelCol='rating',
predictionCol='prediction',
metricName='rmse')

rmse=evaluator.evaluate(prediction)
print("Test rmse score = %g"% rmse)

Test rmse score = 0.923145


### RECOMMENDING MOVIE FOR EVERY USER IN THE DATABASE

In [122]:
userRecsAll=model.recommendForAllUsers(5)


In [123]:
userRecsAll.toPandas().head(3)

,userID,recommendations
0,471,"[(83318, 4.950439453125), (83411, 4.9504394531..."
1,463,"[(83318, 4.943396091461182), (67504, 4.9433960..."
2,496,"[(6413, 5.719137191772461), (3414, 5.501722812..."


VIEW THE USERS WHO ARE MOST LIKELY TO LIKE A MOVIE

In [124]:
from pyspark.sql.types import IntegerType

userList=[148,469,287]
usersDF=spark.createDataFrame(userList,IntegerType()).toDF('userID')

usersDF.take(3)

[Row(userID=148), Row(userID=469), Row(userID=287)]

In [125]:
userRecs=model.recommendForUserSubset(usersDF,5)
userRecs.toPandas()

,userID,recommendations
0,148,"[(8933, 5.0930094718933105), (6413, 5.08081102..."
1,287,"[(735, 6.026994705200195), (2563, 5.8587794303..."
2,469,"[(80, 5.044060230255127), (1085, 5.03934335708..."


In [126]:
userMoviesList=userRecs.filter(userRecs.userID==287).select('recommendations')
userMoviesList.collect()

[Row(recommendations=[Row(movieID=735, rating=6.026994705200195), Row(movieID=2563, rating=5.858779430389404), Row(movieID=3414, rating=5.822144985198975), Row(movieID=390, rating=5.796353340148926), Row(movieID=4518, rating=5.745043754577637)])]

In [127]:
#What is the difference between above representation and this representation

moviesList=userMoviesList.collect()[0].recommendations
moviesList

[Row(movieID=735, rating=6.026994705200195),
 Row(movieID=2563, rating=5.858779430389404),
 Row(movieID=3414, rating=5.822144985198975),
 Row(movieID=390, rating=5.796353340148926),
 Row(movieID=4518, rating=5.745043754577637)]

In [128]:
moviesDF=spark.createDataFrame(moviesList)
moviesDF.toPandas()

,movieID,rating
0,735,6.026995
1,2563,5.858779
2,3414,5.822145
3,390,5.796353
4,4518,5.745044


In [130]:
moviesData=spark.read\
.format('csv')\
.option('header','true')\
.load("/home/titan/dataset/ML_JRVI/movielens/movies.csv")




moviesData=moviesData.withColumnRenamed("movieId","movieID")

moviesData.toPandas().head(3)

,movieID,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [131]:
# # moviesData.(columns={'movieId':'movieID'})...Doesnot work

# moviesData.toPandas().head(3)


In [132]:
recommendedMovies=moviesData.join(moviesDF,on=['movieID'])\
.orderBy('rating',ascending=False)\
.select('title','genres','rating')
recommendedMovies.toPandas()

,title,genres,rating
0,Cemetery Man (Dellamorte Dellamore) (1994),Horror,6.026995
1,Dangerous Beauty (1998),Drama,5.858779
2,Love Is a Many-Splendored Thing (1955),Drama|Romance|War,5.822145
3,Faster Pussycat! Kill! Kill! (1965),Action|Crime|Drama,5.796353
4,The Lair of the White Worm (1988),Comedy|Horror,5.745044


### combining all method for predicting movies for all users

In [139]:
from pyspark.sql.types import IntegerType

def getRecommendationsForUser(userID, numRecs):
    userDFs=spark.createDataFrame([userID],IntegerType()).toDF('userID')
    userRecs=model.recommendForUserSubset(userDFs,numRecs)
    
    moviesLists=userRecs.collect()[0].recommendations
#     moviesLists
    moviesDFs=spark.createDataFrame(moviesLists)
#     moviesDFs.toPandas()
    recommendedMoviess=moviesData.join(moviesDFs,on=['movieID'])\
    .orderBy('rating',ascending=False)\
    .select('title','genres','rating')
    
    return recommendedMoviess

In [140]:
recommendationForUsers=getRecommendationsForUser(234,3)
recommendationForUsers.toPandas()

,title,genres,rating
0,Excision (2012),Crime|Drama|Horror|Thriller,5.111301
1,Land of Silence and Darkness (Land des Schweig...,Documentary,4.956741
2,Cops (1922),Comedy,4.956741


 	Cemetery Man (Dellamorte Dellamore) (1994) 	Horror 	6.026995
1 	Dangerous Beauty (1998) 	Drama 	5.858779
2 	Love Is a Many-Splendored Thing (1955) 	Drama|Romance|War 	5.822145
3 	Faster Pussycat! Kill! Kill! (1965) 	Action|Crime|Drama 	5.796353
4 	The Lair of the White Worm (1988) 	Comedy|Horror 	5.745044